{<a href = "https://colab.research.google.com/drive/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx">Click here to read this notebook in Google Colab</a>}

<body><table>
    <th align = "left">EPAT Batch 45 | Final Project, July 2020</th>
    <th align = "right"><i>Written by: Gaston Solari Loudet</i></th>
</table><hr>

<h3>"Timer" class</h3>

After thorough research through different libraries related to time measurement and event handling, I've realized that there are not many implemented solutions for running tasks in the background at one constant frequency. We'll need this for different reasons. Some examples:

<ul><li><u>Data retrieval</u>: Execution of MT5 data request functions at a certain frequency.
<br>(Somewhat lower than tick data request, but higher than minutely)</li>
<li><u>Trade monitoring</u>: Position analysis according to market's actual situation.
<br>(Useful for tasks such as trailing stops, delay measurement or circuit breaking)</li>
<li><u>Connection monitoring</u>: Measuring of variables such as bandwidth or MT5 comm speed.</li></ul>

The closest thing to it, is the "<a href = "https://docs.python.org/3/library/threading.html"><code>threading</code></a>" library, commonly used as a basic parallel computing tool. Python is able to generate different operating "threads" that run independent from each other. Care must be taken when two threads interact requesting data from each other, as:

<ul><li>Data may be incomplete as a thread may have been unable to complete its task.</li>
<li>A busy thread can "become distracted" while sending data, increasing delays in its work.</li></ul>

The problem is that classes from "<code>threading</code>" package don't hold any method able to reset the execution once it's done: objects such as "<code>Thread</code>" or "<code>Timer</code>" only schedule tasks for future occasions to happen once.</body>

<body><h4>Imports</h4>

We may re-use the aforementioned objects as "<code>threading</code>" package is commonly already installed in most Python environments. We could just create a "children" class that inherits attributes from the original class with the same name, and add the appropriate feats to suit our needs.

We shall also include "<code>time</code>" functions, as time measurements will be essential for measuring deviations from, say, "punctuality". It's better to leave "<code>datetime</code>" aside in this occasion for 2 reasons:

<ul><li>"<code>timedelta</code>" formatting consumes much more memory, and could amplify unnecessary delays.</li>
<li>Its precision is on the order of microseconds at max, and we need to measure in a CPU timeframe.</li></ul></body>

In [11]:
import threading, time

<body><h4>Constructor</h4>

The concept is largely based in two fundamental elements: a "<code>function</code>" to take place, and a time "<code>interval</code>" (step) between 2 consecutive executions. However, function may have its own arguments, isn't it? But we can't include keywords for them in the constructor: functions may vary, and arguments may not be the same.

Luckily, Python has quite a few feats to be taken advantage of:

<ul><li>Functions with unspecified arguments are callback methods, able to be manipulated as objects (like "<code>lambda</code>").
<br>We can call function "<code>f</code>" without having to read a line containing "<code>f(x, y,...)</code>" or with eval-type functions.</li>
<li>Retrieving optional input arguments by unpacking "<code>args</code>" and "<code>kwargs</code>" keywords with "*" (list) and "**" (dict) respectively.</li></ul>

We've already mentioned 4 things to be stored by a "<code>timer</code>" instance: "<code>interval</code>", "<code>function</code>", "<code>args</code>" and "<code>kwargs</code>". We need to give a primordial "<code>threading.Timer</code>" object to it too. Take notice that as it's intended to be manipulated from upcoming instance methods and not from its attribute directly, we shall label our "<code>_timer</code>" it with an underscore.

I will add 3 more variables to be explained later: "<code>is_running</code>" (boolean), "<code>then</code>" (float) and "<code>delays</code>" (list of floats).</body>

In [12]:
class timer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self.interval, self.function, self.args, self.kwargs = interval, function, args, kwargs
        self._timer, self.is_running, self.then, self.delays = None, False, time.time(), []

<body><h4>Principle</h4>

The main problem with "<code>threading.Timer</code>" is that its timing function ceases to exist once its associated function has already run. We will need to re-create the object shall the function execute. But the function "def" doesn't have the timer creation on the inside.

We'll then need to create some "<code>start</code>" method whose role would be essentially to execute the "<code>function</code>" as callback with its arguments (unpacked "<code>args</code>" and "<code>kwargs</code>") and then make the timer to start again. The "<code>_timer</code>" shall run <b>this</b> method as callback, not the "<code>function</code>" so as not to repeat the problem with "<code>threading</code>".

We shall control the timer activity with the "<code>is_running</code>" flag. When it is "<code>True</code>", timer shall be able to become reset after every loop. Otherwise, function wouldn't do anything.

On the other hand, we shall add the required time measurement features:
<ol><li>Before the upcoming timer reset, we shall take note of the actual "<code>time</code>" as "<code>then</code>".</li>
<li>On the next cycle, the "<code>start</code>" and the actual "<code>function</code>" have been re-executed:
<br>...we shall record the difference between the "new" present and "<code>then</code>".</li>
<li>The difference shall be stored in a list of "<code>delays</code>" with no more than the last 100 records.
<br>...this list can be useful for later comparison.</li></ol></body>

In [13]:
class timer(timer):
    def __init__(self, interval, function, *args, **kwargs):
        super().__init__(interval, function, *args, **kwargs)
    def start(self):
        if self.is_running:
            self.function(*self.args, **self.kwargs)
            self.delays.append(time.time() - self.then)
            if (len(self.delays) > 100): self.delays.pop(0)
            self.then = time.time()
            self._timer = threading.Timer(self.interval, self.start)
            self._timer.start() # Switch "threading.Timer" instance on.

<body>Note that a "<code>threading.Timer</code>" needs its own "<code>start</code>" command to begin counting time until run. This utility must end with such call. On the other hand, we need a different method for the opposite action: it's not enough just to unset "<code>is_running</code>" from the outside because the original timer object needs a "<code>cancel</code>" command to stop as well. Our next "<code>stop</code>" method will take care of this.</body>

In [14]:
class timer(timer):
    def __init__(self, interval, function, *args, **kwargs):
        super().__init__(interval, function, *args, **kwargs)
    def stop(self):
        self._timer.cancel()
        self.is_running = False

<body>As an additional feat to ease the testing, we could add the following dunder to be able to run and stop the timer by only typing its name in the command line. It shall return the mean of all the core function's execution times.</body>

In [15]:
class timer(timer):
    def __init__(self, interval, function, *args, **kwargs):
        super().__init__(interval, function, *args, **kwargs)
    def __repr__(self):
        if self.is_running: self.stop()
        else: self.is_running = True; self.start()
        return str(sum(self.delays)/len(self.delays))

<body><h4>Unit tests</h4>

Our timer class construction is now complete. Now we shall write a few trial functions for correct manipulation and execution time measurement. Let's choose 3 different examples with varying execution times to see how much does the timer delay in its reset because of running the particular "<code>function</code>" itself:
    
<ol><li>"Easy" (about 30 us): function just "<code>return</code>s" and does nothing else.</li>
<li>"Medium" (about 4500 us): "<code>print</code>s" the actual time in "<code>time</code>" library format.</li>
<li>"Hard" (about 90000 us): given a large initial "$p$", it will repeatedly calculate "$p^{0.999}$".</li></ol>
Such times will <b>largely</b> depend on factors such as computer's CPU/OS/RAM, if other tasks are being done, etc.
<br>Also times will likely be larger with timers being executed here as ".ipynb" files are more complex than ".py".</body>

In [16]:
def timer_test_easy():
    def f(): return
    return timer(interval = 1/10, function = f)
def timer_test_medium():
    def f(): print(time.time())
    return timer(interval = 1/10, function = f)
def timer_test_hard():
    def f():
        p = 1e100
        for n in range(100000): p = p**0.999
    return timer(interval = 1/10, function = f)

<body><h4>Conclusions</h4>

The class has successfully accomplished its role, and will be applied in future requirements.
<br>There are a few considerations to bear in mind:

<ol><li>After some experimentation, I found the delay to be <b>independent</b> on the selected "<code>interval</code>" itself, in all cases above.<br>Luckily, this grants us the freedom to adjust the timer frequency while only considering the provided "<code>function</code>" runtime.</li>
<li>Let "$\Delta t_{del}$" be the average delay, and "$\Delta t_{int}$" the selected "<code>interval</code>". If "$\Delta t_{int} / \Delta t_{del} = n$", that means that the system will erroneously skip/omit one execution after around "$n$" ones. Logically, "$n$" must be as large as possible.</li></ol></body>

In [17]:
x = timer_test_medium()